In [1]:
%matplotlib inline

In [2]:
!pip install lightly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=613950826eca1256b3c733c60fec40014d6dc7332957337abe549c75281a59f7
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
Successfully built antlr4-python3-runtime


In [3]:
import torch
#import torch.nn as nn
import torchvision
import pytorch_lightning as pl
import lightly
#from torchmetrics import F1
from torchmetrics.classification import Accuracy

In [4]:
!nvidia-smi

Thu Dec 29 11:02:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import torch
torch.__version__

'1.11.0'

In [6]:
from torchvision.models import resnet101
from torch import nn
from torch.utils.data import DataLoader

In [7]:
import torchvision
from torchvision import datasets 
from torchvision.transforms import ToTensor

In [8]:
path_to_train ='/kaggle/input/augmented-knee-ap-implants/Augmented Knee AP Dataset/Train'
path_to_test ='/kaggle/input/augmented-knee-ap-implants/Augmented Knee AP Dataset/Val'

In [9]:
train_classifier_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((400,400)),
    #torchvision.transforms.RandomHorizontalFlip(p=0.5),
    #torchvision.transforms.RandomRotation((-40,40)),
    #torchvision.transforms.Normalize(
           #mean=[0.485, 0.456, 0.406],
            #std=[0.229, 0.224, 0.225]
        #)
])
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((400, 400)),
    #torchvision.transforms.Normalize(
           #mean=[0.485, 0.456, 0.406],
            #std=[0.229, 0.224, 0.225]
        #)
])

In [10]:
train_set = datasets.ImageFolder(root=path_to_train, transform= train_classifier_transforms)
test_set = datasets.ImageFolder(root= path_to_test , transform = test_transforms)

In [11]:
train_dl = DataLoader(train_set, batch_size=16, shuffle=True)
test_dl = DataLoader(test_set, batch_size=16)

In [12]:
import pytorch_lightning as pl

In [13]:
class ResNetMNIST(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = resnet101(num_classes=6)
    self.loss = nn.CrossEntropyLoss()
    self.accuracy = Accuracy()

  def forward(self, x):
    return self.model(x)
  
  def training_step(self, batch, batch_no):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y)
    
    preds = torch.argmax(logits, dim = 1)
    acc = self.accuracy(preds, y)
    self.log('train_acc', acc,
             on_epoch=True, prog_bar=True)
    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y)
    # validation metrics
    preds = torch.argmax(logits, dim=1)
    acc = self.accuracy(preds, y)
    self.log('val_loss', loss, prog_bar=True)
    self.log('val_acc', acc, prog_bar=True)
    return loss

  def configure_optimizers(self):
    return torch.optim.RMSprop(self.parameters(), lr=0.05)

In [14]:
model = ResNetMNIST()

In [15]:
trainer = pl.Trainer(
    gpus=1,
    max_epochs=50,
    #progress_bar_refresh_rate=20
)
trainer.fit(model, train_dl, test_dl)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [16]:
trainer.save_checkpoint("squeezenet1_0_mnist.pt")

In [17]:
def get_prediction(x, model: pl.LightningModule):
  model.freeze() # prepares model for predicting
  probabilities = torch.softmax(model(x), dim=1)
  predicted_class = torch.argmax(probabilities, dim=1)
  return predicted_class, probabilities

In [18]:
from tqdm.autonotebook import tqdm

In [19]:
inference_model = ResNetMNIST.load_from_checkpoint("./squeezenet1_0_mnist.pt")

In [20]:
true_y, pred_y = [], []
for batch in tqdm(iter(test_dl), total=len(test_dl)):
  x, y = batch
  true_y.extend(y)
  preds, probs = get_prediction(x, inference_model)
  pred_y.extend(preds.cpu())

  0%|          | 0/11 [00:00<?, ?it/s]

In [21]:
from sklearn.metrics import classification_report

In [22]:
print(classification_report(true_y, pred_y, digits=2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      0.91      0.95        34
           2       0.88      1.00      0.93        21
           3       1.00      1.00      1.00        21
           4       1.00      0.90      0.95        20
           5       0.94      1.00      0.97        34

    accuracy                           0.97       163
   macro avg       0.97      0.97      0.97       163
weighted avg       0.97      0.97      0.97       163



In [23]:
#a.shape

In [24]:
#b= cv2.imread('../input/mnist-39k/MNIST-Data-BW&Color-39k-BW/Val/0/slice_1002.png')

In [25]:
#b.shape

In [26]:
#import matplotlib.pyplot as plt

In [27]:
#plt.hist(a)